In [34]:
import json
import json5
import demjson

In [60]:
with open('../../data/australian_user_reviews.json', 'r') as f:
    user_reviews = demjson.decode(f.read())


In [69]:
user_reviews[0]

{'user_id': '76561197970982479',
 'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
 'reviews': [{'funny': '',
   'posted': 'Posted November 5, 2011.',
   'last_edited': '',
   'item_id': '1250',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
  {'funny': '',
   'posted': 'Posted July 15, 2011.',
   'last_edited': '',
   'item_id': '22200',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': "It's unique and worth a playthrough."},
  {'funny': '',
   'posted': 'Posted April 21, 2011.',
   'last_edited': '',
   'item_id': '43110',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this 

In [61]:

with open('../../data/australian_users_items.json', 'r') as f:
    user_items = demjson.decode(f.read())


In [68]:
user_items[0]

{'user_id': 'Wackky',
 'items_count': 0,
 'steam_id': '76561198039117046',
 'user_url': 'http://steamcommunity.com/id/Wackky',
 'items': []}

In [62]:
with open('../../data/steam_games.json', 'r') as f:
    steam_games = demjson.decode(f.read())

# with open('../../data/steam_review.json', 'r') as f:
#     steam_reviews = demjson.decode(f.read())

In [65]:
steam_games[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [ ]:
# {'publisher': 'Kotoshiro',
#  'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
#  'app_name': 'Lost Summoner Kitty',
#  'title': 'Lost Summoner Kitty',
#  'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
#  'release_date': '2018-01-04',
#  'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
#  'discount_price': 4.49,
#  'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
#  'specs': ['Single-player'],
#  'price': 4.99,
#  'early_access': False,
#  'id': '761140',
#  'developer': 'Kotoshiro'}
        

game_details = {
    game.get('id', None): {
        'developer': game.get('developer', None),
        'tags': game.get('tags', None)
        # 'app_name': game.get('app_name', None),
        # 'price': game.get('price', None),
        # 'genres': game.get('genres', None),
        # 'discount_price': game.get('discount_price', None)
        # 'specs': game.get('specs', None),
        # 'early_access': game.get('early_access', None),
        # 'url': game.get('url', None),
        # 'publisher': game.get('publisher', None),
        # 'reviews_url': game.get('reviews_url', None)
    } for game in steam_games if game.get('id') is not None 
}



user_data = {}
user_game_reviews = {}
user_data_map = {}

for user in user_reviews:
    user_id = user['user_id']
    user_data[user_id] = {
        # 'user_url': user['user_url'],
        'reviews': []
    }

    for review in user['reviews']:
        game_id = review['item_id']
        game_info = game_details.get(game_id, {})
        user_data[user_id]['reviews'].append({
            'game_id': game_id,
            'posted': review['posted'],
            'recommend': review['recommend']
            # 'tags': game_info.get('tags', []),
            # 'review': review['review'],
            # 'game_name': game_info.get('app_name', 'Unknown'),
            # 'price': game_info.get('price', 0),
            # 'discount_price': game_info.get('discount_price', 0),
            # 'url': game_info.get('url', '')
        })

for user in user_reviews:
    user_id = user['user_id']
    user_data_map[user_id] = {
        # 'user_url': user['user_url'],
        'reviews': {}
    }

    for review in user['reviews']:
        game_id = review['item_id']
        game_info = game_details.get(game_id, {})
        user_data_map[user_id][game_id] = {
            'posted': review['posted'],
            'recommend': review['recommend']
            # 'tags': game_info.get('tags', []),
            # 'review': review['review'],
            # 'url': game_info.get('url', ''),
            # 'game_name': game_info.get('app_name', 'Unknown'),
            # 'price': game_info.get('price', 0),
            # 'discount_price': game_info.get('discount_price', 0)
        }



for user in user_items:
    user_id = user['user_id']
    if user_id in user_data: 
        user_data[user_id]['items'] = []
        for item in user['items']:
            game_id = item['item_id']
            game_info = game_details.get(game_id, {})
            user_data[user_id]['items'].append({
                'game_id': game_id,
                'playtime_forever': item['playtime_forever'],
                # 'price': game_info.get('price', 0)
                # 'tags': game_info.get('tags', []),
                # 'playtime_2weeks': item['playtime_2weeks'],
                # 'game_name': game_info.get('app_name', 'Unknown'),
                # 'discount_price': game_info.get('discount_price', 0),
                # 'url': game_info.get('url', '')
            })


In [74]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

def get_game_overall_rating(game_id):
    reviews = user_data.get(game_id, {}).get('reviews', [])
    if len(reviews) == 0:
        return 0
    return sum(1 for r in reviews if r['recommend']) / len(reviews)


X_combined = []
y_combined = []

for user_id, user_info in user_data.items():
    for review in user_info.get('reviews', []):
        game_id = review['game_id']
        playtime = next((item['playtime_forever'] for item in user_info.get('items', []) if item['game_id'] == game_id), 0)
        tags = game_details.get(game_id, {}).get('tags', [])
        developer = game_details.get(game_id, {}).get('developer', 'Unknown')
        overall_rating = get_game_overall_rating(game_id)  
        user_avg_rating = sum(1 for r in user_info.get('reviews', []) if r['recommend']) / len(user_info.get('reviews', [])) if len(user_info.get('reviews', [])) > 0 else 0

        # One-Hot Encoding tags
        mlb = MultiLabelBinarizer()
        tags_encoded = mlb.fit_transform([tags])[0]

        # Label Encoding developer
        le = LabelEncoder()
        developer_encoded = le.fit_transform([developer])[0]

        X_combined.append([playtime] + tags_encoded.tolist() + [developer_encoded, overall_rating, user_avg_rating])

        y_combined.append(1 if review['recommend'] else 0)

X_combined = np.array(X_combined)
y_combined = np.array(y_combined)

X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))



ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2451,) + inhomogeneous part.

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

model1 = LogisticRegression()
model2 = RandomForestClassifier()
model3 = SVC()

voting_clf = VotingClassifier(estimators=[('lr', model1), ('rf', model2), ('svc', model3)], voting='hard')

voting_clf.fit(X_train, y_train)

y_pred_voting = voting_clf.predict(X_test)
print(classification_report(y_test, y_pred_voting))
